In [1]:
import torch
import os
import re
import torch
import random
import numpy as np
from collections import defaultdict, Counter
import tqdm
# from torch_geometric.datasets import DBLP

In [2]:
# data = torch.load('data/processed/data.pt')
# print(data)

(HeteroData(
  author={
    x=[4057, 334],
    y=[4057],
    train_mask=[4057],
    val_mask=[4057],
    test_mask=[4057]
  },
  paper={ x=[14328, 4231] },
  term={ x=[7723, 50] },
  conference={ num_nodes=20 },
  (author, to, paper)={ edge_index=[2, 19645] },
  (paper, to, author)={ edge_index=[2, 19645] },
  (paper, to, term)={ edge_index=[2, 85810] },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (term, to, paper)={ edge_index=[2, 85810] },
  (conference, to, paper)={ edge_index=[2, 14328] }
), None)


In [3]:
# a_data = data[0]['author'].x
# a_label = data[0]['author'].y
# p_data = data[0]['paper'].x
# t_data = data[0]['term'].x

# c_num = data[0]['conference'].num_nodes
# ap_edges = data[0]['author','to','paper'].edge_index
# pa_edges = data[0]['paper','to','author'].edge_index
# pt_edges = data[0]['paper','to','term'].edge_index
# tp_edges = data[0]['term','to','paper'].edge_index
# pc_edges = data[0]['paper','to','conference'].edge_index
# cp_edges = data[0]['conference','to','paper'].edge_index

# print(a_data)
# print(a_label)
# print(p_data)
# print(t_data)
# print(c_num)
# print(ap_edges)
# print(pa_edges)
# print(pt_edges)
# print(tp_edges)
# print(pc_edges)
# print(cp_edges)

tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([2, 2, 3,  ..., 0, 0, 0])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[-0.6924, -0.4659,  1.1540,  ...,  0.9178,  0.1995, -0.6360],
        [ 1.2031, -0.4003,  0.0740,  ...,  1.3262, -0.3325,  0.8198],
        [ 0.3748,  0.5731,  0.4802,  ...,  1.1522,  0.6010, -0.4309],
        ...,
        [ 0.4180,  0.2497, -0.4124,  ..., -0.1841, -0.1151, -0.7858],
        [ 0.1724, -0.2723, -1.3368,  ..., -0.0881,  0.0225,  0.1166],
        [ 0.2197,  0.0253,  0.1220,  ...,  0.0871, -0.5351, -0.4949]])
20
tensor([[    0,     0,

In [4]:
# # Neighbour reader for all nodes
# neigh_dict = defaultdict(list)
# edge_lists = [ap_edges, tp_edges, cp_edges, pa_edges, pt_edges,  pc_edges]

# for i, edge_list in enumerate(edge_lists):
    
#     for col in range(edge_list.size(1)):
#         if i == 0:
#             neigh_dict[f'a{edge_list[0][col]}'].append(f'p{edge_list[1][col]}')
#         elif i == 1:
#             neigh_dict[f't{edge_list[0][col]}'].append(f'p{edge_list[1][col]}')
#         elif i == 2:
#             neigh_dict[f'c{edge_list[0][col]}'].append(f'p{edge_list[1][col]}')
#         else: 
#             if i == 3: n = 'a'
#             elif i == 4: n = 't'
#             else: n = 'c'
#             neigh_dict[f'p{edge_list[0][col]}'].append(f'{n}{edge_list[1][col]}')

In [5]:
# print(neigh_dict)

In [6]:
# # Random path maker
# random_walks = defaultdict(list)

# for node in list(neigh_dict.keys()):
#     # print(node)
#     curNode = node
    
#     walk_size, a_size, p_size, t_size, c_size = 0,0,0,0,0
    
#     while walk_size < 160:
#         prob =  random.random()
#         if prob < 0.5:
#             curNode = node
#         else:
#             # if curNode not in list(neigh_dict.keys()):
#             #     print(node, curNode)
#             # print(neigh_dict[curNode])
#             curNode = random.choice(neigh_dict[curNode])
#             if curNode != node:
#                 if curNode[0] == 'a' and a_size < 60:
#                     random_walks[node].append(curNode)
#                     a_size += 1
#                 elif curNode[0] == 'p' and p_size < 90:
#                     random_walks[node].append(curNode)
#                     p_size += 1
#                 elif curNode[0] == 't' and t_size < 60:
#                     random_walks[node].append(curNode)
#                     t_size += 1
#                 elif curNode[0] == 'c' and c_size < 10:
#                     random_walks[node].append(curNode)
#                     c_size += 1
                
#                 walk_size += 1

In [7]:
# with open('data/het_neigh_train.txt', 'w') as file:
#     for node in list(random_walks.keys()):
#         file.write(node + ':' + ','.join(random_walks[node])+'\n')

In [8]:
# # node net embedding generator
# from gensim.models import Word2Vec
# from itertools import *
# dimen = 128
# window = 5

# def read_random_walk_corpus():
#     walks, node_ids = [], []
#     with open("data/het_neigh_train.txt", 'r') as file:
#         lines = file.readlines()
#         for i, line in enumerate(lines):
#             parts = line.strip().split(':')
#             node_ids.append(parts[0])
#             path = parts[1].split(',')
#             walks.append(path)

#     return walks, node_ids

# walk_corpus, node_ids = read_random_walk_corpus()

# # Train the Word2Vec model
# model = Word2Vec(walk_corpus, vector_size=dimen, window=window, min_count=0, workers=8, sg=1, hs=0, negative=5)

# # Save node embeddings to a text file
# with open("node_net_embedding.txt", 'w') as file:
#     for i, node_id in enumerate(node_ids):
#         embedding = model.wv[node_id]
#         embedding_str = " ".join(str(val) for val in embedding)
#         file.write(node_id + " " + embedding_str + "\n")
# model.wv.save_word2vec_format("data/node_net_embedding.txt")

In [9]:
# a_edge_list = []
# p_edge_list = []
# t_edge_list = []
# c_edge_list = []
# a_a_edge_index = []
# a_p_edge_index = []
# a_t_edge_index = []
# a_c_edge_index = []
# p_a_edge_index = []
# p_p_edge_index = []
# p_t_edge_index = []
# p_c_edge_index = []
# t_a_edge_index = []
# t_p_edge_index = []
# t_t_edge_index = []
# t_c_edge_index = []
# c_a_edge_index = []
# c_p_edge_index = []
# c_t_edge_index = []
# c_c_edge_index = []

# with open("data/het_neigh_train.txt", 'r') as file:            
#     lines = file.readlines()
        
# for i, line in enumerate(lines):
#     line = line.strip()
#     node_type = re.split(':', line)[0][0]
#     node_id = int(re.split(':', line)[0][1:])
#     neigh_list = re.split(',', re.split(':', line)[1].strip())
    
#     a_edge_list = [node for node in neigh_list if node.startswith('a')]
#     p_edge_list = [node for node in neigh_list if node.startswith('p')]
#     t_edge_list = [node for node in neigh_list if node.startswith('t')]
#     c_edge_list = [node for node in neigh_list if node.startswith('c')]
    
#     # Count the frequency of elements starting with 'r', 'u', and 'b'
#     a_counts = Counter(a_edge_list)
#     p_counts = Counter(p_edge_list)
#     t_counts = Counter(t_edge_list)
#     c_counts = Counter(c_edge_list)
#     # print(a_counts)
    
#     a_edge_list = [node for node, count in a_counts.most_common(8)]
#     p_edge_list = [node for node, count in p_counts.most_common(10)]
#     t_edge_list = [node for node, count in t_counts.most_common(8)]
#     c_edge_list = [node for node, count in c_counts.most_common(3)]
#     # print(a_edge_list)
    
#     if node_type == 'a':
#         a_a_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in a_edge_list])
#         a_p_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in p_edge_list])
#         a_t_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in t_edge_list])
#         a_c_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in c_edge_list])
#     elif node_type == 'p':
#         p_a_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in a_edge_list])
#         p_p_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in p_edge_list])
#         p_t_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in t_edge_list])
#         p_c_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in c_edge_list])
#     elif node_type == 't':
#         t_a_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in a_edge_list])
#         t_p_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in p_edge_list])
#         t_t_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in t_edge_list])
#         t_c_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in c_edge_list])
#     else:
#         c_a_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in a_edge_list])
#         c_p_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in p_edge_list])
#         c_t_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in t_edge_list])
#         c_c_edge_index.extend([torch.tensor([[node_id, int(node[1:])]]) for node in c_edge_list])

# # Concatenate the list of tensors into a single tensor
# a_a_edge_index = torch.cat(a_a_edge_index, dim=0).t().contiguous()
# a_p_edge_index = torch.cat(a_p_edge_index, dim=0).t().contiguous()
# a_t_edge_index = torch.cat(a_t_edge_index, dim=0).t().contiguous()
# a_c_edge_index = torch.cat(a_c_edge_index, dim=0).t().contiguous()
# p_a_edge_index = torch.cat(p_a_edge_index, dim=0).t().contiguous()
# p_p_edge_index = torch.cat(p_p_edge_index, dim=0).t().contiguous()
# p_t_edge_index = torch.cat(p_t_edge_index, dim=0).t().contiguous()
# p_c_edge_index = torch.cat(p_c_edge_index, dim=0).t().contiguous()
# t_a_edge_index = torch.cat(t_a_edge_index, dim=0).t().contiguous()
# t_p_edge_index = torch.cat(t_p_edge_index, dim=0).t().contiguous()
# t_t_edge_index = torch.cat(t_t_edge_index, dim=0).t().contiguous()
# t_c_edge_index = torch.cat(t_c_edge_index, dim=0).t().contiguous()
# c_a_edge_index = torch.cat(c_a_edge_index, dim=0).t().contiguous()
# c_p_edge_index = torch.cat(c_p_edge_index, dim=0).t().contiguous()
# c_t_edge_index = torch.cat(c_t_edge_index, dim=0).t().contiguous()
# c_c_edge_index = torch.cat(c_c_edge_index, dim=0).t().contiguous()

KeyboardInterrupt: 

In [ ]:
# embed_d = 128
# A_n, P_n, T_n, C_n = 4057, 14328, 7723, 20
# a_embed = torch.zeros(A_n, embed_d, dtype=torch.float32)
# p_embed = torch.zeros(P_n, embed_d, dtype=torch.float32)
# t_embed = torch.zeros(T_n, embed_d, dtype=torch.float32) 
# c_embed = torch.zeros(C_n, embed_d, dtype=torch.float32) 

# with open("data/node_net_embedding.txt", 'r') as file:
#     lines =  file.readlines()
    
# for i, line in enumerate(lines):
#     entries = line.strip().split()
#     node_id = int(entries[0][1:])
#     node_type = entries[0][0]
#     embed_list = entries[1:]
    
#     if node_type == 'a':
#         a_embed[node_id] = torch.tensor([float(value) for value in embed_list], dtype=torch.float32)
#     elif node_type == 'p':
#         p_embed[node_id] = torch.tensor([float(value) for value in embed_list], dtype=torch.float32)
#     elif node_type == 't':
#         t_embed[node_id] = torch.tensor([float(value) for value in embed_list], dtype=torch.float32)
#     elif node_type == 'c':
#         c_embed[node_id] =  torch.tensor([float(value) for value in embed_list], dtype=torch.float32)

In [ ]:
# from torch_geometric.data import HeteroData
# data2 = HeteroData()

# data2['a'].num_nodes = A_n
# data2['p'].num_nodes = P_n
# data2['t'].num_nodes = T_n
# data2['c'].num_nodes = C_n

# data2['a_embed'].x = a_embed
# data2['p_embed'].x = p_embed
# data2['t_embed'].x = t_embed
# data2['c_embed'].x = c_embed
    
# data2['a'].x = a_data
# data2['p'].x = p_data
# data2['t'].x = t_data

# data2['a'].y = a_label

# data2['a', 'walk', 'a'].edge_index = a_a_edge_index
# data2['a', 'walk', 'p'].edge_index = a_p_edge_index
# data2['a', 'walk', 't'].edge_index = a_t_edge_index
# data2['a', 'walk', 'c'].edge_index = a_c_edge_index

# data2['p', 'walk', 'a'].edge_index = p_a_edge_index
# data2['p', 'walk', 'p'].edge_index = p_p_edge_index
# data2['p', 'walk', 't'].edge_index = p_t_edge_index
# data2['p', 'walk', 'c'].edge_index = p_c_edge_index

# data2['t', 'walk', 'a'].edge_index = t_a_edge_index
# data2['t', 'walk', 'p'].edge_index = t_p_edge_index
# data2['t', 'walk', 't'].edge_index = t_t_edge_index
# data2['t', 'walk', 'c'].edge_index = t_c_edge_index

# data2['c', 'walk', 'a'].edge_index = c_a_edge_index
# data2['c', 'walk', 'p'].edge_index = c_p_edge_index
# data2['c', 'walk', 't'].edge_index = c_t_edge_index
# data2['c', 'walk', 'c'].edge_index = c_c_edge_index

# data2['a','to','p'].edge_index = ap_edges
# data2['p','to','a'].edge_index = pa_edges
# data2['p','to','t'].edge_index = pt_edges
# data2['t','to','p'].edge_index = tp_edges
# data2['p','to','c'].edge_index = pc_edges
# data2['c','to','p'].edge_index = cp_edges

In [ ]:
# torch.save(data2, "data/dblp_data.pt")

In [ ]:
# new_data = torch.load('data/dblp_data.pt')
# print(new_data)